In [21]:
from sklearn.model_selection import LeaveOneGroupOut
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score

In [43]:
train_df = pd.read_csv('../data/processed/train.csv')
test_df = pd.read_csv('../data/processed/test.csv')

In [ ]:
X_train = train_df.drop(['state', 'name'], axis=1)
y_train = train_df['state']
X_test = test_df.drop(['state', 'name'], axis=1)
y_test = test_df['state']

# X_all = X_train.append(X_test)
# y_all = y_train.append(y_test)

groups = train_df['name']

outcomes = []

clf = RandomForestClassifier(criterion='gini',
                             max_features=None,
                             min_samples_split=5,
                             n_estimators=100)

print(len(groups))
def run_pseudo_labeling(clf):
    logo = LeaveOneGroupOut()
    group = 0
    
    labeled_index = np.empty(0, int)
    unlabeled_index = np.empty(0, int)
    
    for train_index, test_index in logo.split(X_train, groups=groups):
        labeled_index = np.append(labeled_index, test_index)
        unlabeled_index = np.array(list(set(X_train.index.values).symmetric_difference(labeled_index)))
        
        X_labeled = X_train.values[labeled_index]
        y_labeled = y_train.values[labeled_index]
        
        print(len(X_labeled))

        clf.fit(X_labeled, y_labeled)
        predictions = clf.predict(X_test)
        
        print(classification_report(y_test, predictions))
        print(confusion_matrix(y_test, predictions))
        
        accuracy = balanced_accuracy_score(y_test, predictions)
#         outcomes.append(accuracy)
        print("Group {0} accuracy: {1}".format(group, accuracy))
    
        # Predict and add unlabeled samples to training set
        predictions = clf.predict(X_train.values[unlabeled_index])
        y_augmented = y_train.copy(deep=True)
        y_augmented.values[unlabeled_index] = predictions
        
        clf.fit(X_train, y_augmented)
        predictions = clf.predict(X_test)
        
        print(classification_report(y_test, predictions))
        print(confusion_matrix(y_test, predictions))
        
        accuracy = balanced_accuracy_score(y_test, predictions)
#         outcomes.append(accuracy)
        print("Group {0} accuracy: {1}".format(group, accuracy))
    
        group += 1
#     mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

run_pseudo_labeling(clf)

21418
6093
              precision    recall  f1-score   support

           0       0.92      0.93      0.92      2566
           1       0.88      0.94      0.91      2661
           2       0.73      0.60      0.66       395
           3       0.81      0.51      0.62       411

   micro avg       0.88      0.88      0.88      6033
   macro avg       0.83      0.75      0.78      6033
weighted avg       0.88      0.88      0.88      6033

[[2395  111   14   46]
 [ 133 2490   35    3]
 [  69   88  238    0]
 [  20  142   40  209]]
Group 0 accuracy: 0.7450363047390627
              precision    recall  f1-score   support

           0       0.90      0.93      0.92      2566
           1       0.87      0.93      0.90      2661
           2       0.78      0.54      0.64       395
           3       0.77      0.48      0.59       411

   micro avg       0.88      0.88      0.88      6033
   macro avg       0.83      0.72      0.76      6033
weighted avg       0.87      0.88      0.87 

In [97]:
#LOOCV pseudo-labeling woth a separate test set
#sample_rate relative to validate set
from sklearn.model_selection import LeaveOneGroupOut
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score

train_df = pd.read_csv('../data/processed/train.csv')
test_df = pd.read_csv('../data/processed/test.csv')

X_train = train_df.drop(['state', 'name'], axis=1)
y_train = pd.DataFrame(train_df['state'])

X_test = test_df.drop(['state', 'name'], axis=1)
y_test = pd.DataFrame(test_df['state'])

groups = train_df['name']

outcomes = []

def run_pseudo_labeling():
    logo = LeaveOneGroupOut()
    group = 0
    
    for train_index, test_index in logo.split(X_train, groups=groups): 

        X = pd.DataFrame(data=X_train.values[train_index])
        y = pd.DataFrame(data=y_train.values[train_index], columns=['state'])

        # Create the PseudoLabeler with XGBRegressor as the base regressor
        model = PseudoLabeler(
        #    RandomForestClassifier(n_estimators=100),
            GradientBoostingClassifier(n_estimators=100),
            pd.DataFrame(X_train.values[test_index]),
            'state',
            sample_rate=0.0
        )
        
        # Train the model and use it to predict
        model.fit(X, y)
        y_hat = model.predict(X_test)
        
        print(classification_report(y_test, y_hat))
        print(confusion_matrix(y_test, y_hat))        
        accuracy = balanced_accuracy_score(y_test, y_hat)
        print("Group {0} accuracy: {1}".format(group, accuracy))
        
        model.set_params(sample_rate=0.3)
        model.fit(X, y)
        y_hat = model.predict(X_test)
        
        print(classification_report(y_test, y_hat))
        print(confusion_matrix(y_test, y_hat))        
        accuracy = balanced_accuracy_score(y_test, y_hat)
        outcomes.append(accuracy)
        print("Group {0} accuracy: {1}".format(group, accuracy))
    
        group += 1
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

run_pseudo_labeling()

accuracy = balanced_accuracy_score(y_test, y_hat)

(15325, 150)
(15325, 1)
sample rate: 0.0


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.98      0.85      0.91      2566
           1       0.81      0.99      0.89      2661
           2       0.60      0.65      0.62       395
           3       0.88      0.31      0.45       411

   micro avg       0.86      0.86      0.86      6033
   macro avg       0.82      0.70      0.72      6033
weighted avg       0.87      0.86      0.85      6033

[[2188  285   77   16]
 [  10 2622   27    2]
 [  23  117  255    0]
 [  11  207   67  126]]
Group 0 accuracy: 0.697542957286177
(15325, 150)
(15325, 1)
sample rate: 0.3


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.99      0.76      0.86      2566
           1       0.75      0.99      0.85      2661
           2       0.62      0.61      0.61       395
           3       0.89      0.30      0.45       411

   micro avg       0.82      0.82      0.82      6033
   macro avg       0.81      0.67      0.69      6033
weighted avg       0.85      0.82      0.81      6033

[[1953  531   67   15]
 [   1 2640   20    0]
 [  17  138  240    0]
 [  11  216   61  123]]
Group 0 accuracy: 0.6650200051680902
(17501, 150)
(17501, 1)
sample rate: 0.0


...

              precision    recall  f1-score   support

           0       0.92      0.93      0.93      2566
           1       0.87      0.94      0.90      2661
           2       0.69      0.65      0.67       395
           3       0.91      0.47      0.62       411

   micro avg       0.88      0.88      0.88      6033
   macro avg       0.85      0.75      0.78      6033
weighted avg       0.88      0.88      0.88      6033

[[2389  107   52   18]
 [ 139 2491   31    0]
 [  48   88  258    1]
 [  16  168   35  192]]
Group 4 accuracy: 0.7468632822066399
(20297, 150)
(20297, 1)
sample rate: 0.0


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.92      0.92      0.92      2566
           1       0.86      0.93      0.90      2661
           2       0.64      0.63      0.64       395
           3       0.89      0.44      0.59       411

   micro avg       0.87      0.87      0.87      6033
   macro avg       0.83      0.73      0.76      6033
weighted avg       0.88      0.87      0.87      6033

[[2358  134   55   19]
 [ 148 2488   24    1]
 [  37  106  250    2]
 [  15  153   62  181]]
Group 5 accuracy: 0.7318068795676183
(20297, 150)
(20297, 1)
sample rate: 0.3


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.93      0.91      0.92      2566
           1       0.86      0.94      0.90      2661
           2       0.65      0.64      0.64       395
           3       0.88      0.43      0.58       411

   micro avg       0.87      0.87      0.87      6033
   macro avg       0.83      0.73      0.76      6033
weighted avg       0.87      0.87      0.87      6033

[[2335  155   55   21]
 [ 139 2497   25    0]
 [  36  103  254    2]
 [  14  162   59  176]]
Group 5 accuracy: 0.7299018676166621
Mean Accuracy: 0.7215613389858992


In [138]:
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.base import BaseEstimator, RegressorMixin
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

class PseudoLabeler(BaseEstimator, RegressorMixin):
    
    def __init__(self, model, test, target, sample_rate=0.2, seed=42):
        self.sample_rate = sample_rate
        self.seed = seed
        self.model = model
        self.model.seed = seed
        
        self.test = test
        self.target = target
        
    def get_params(self, deep=True):
        return {
            "sample_rate": self.sample_rate,
            "seed": self.seed,
            "model": self.model,
            "test": self.test,
            "target": self.target
        }
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def fit(self, X, y):
        print(X.shape)
        print(y.shape)
        print('sample rate:', self.sample_rate)
        if self.sample_rate > 0.0:
            augmented_X_train, augmented_y_train = self.__create_augmented_train(X, y)
            self.model.fit(
                augmented_X_train,
                augmented_y_train
            )
        else:
            self.model.fit(X, y)
            
        return self
    
    def partial_fit(self, X, y):
        self.model.partial_fit(X, y)
        return self
    
    def __create_augmented_train(self, X, y):
        '''
        Create and return the augmented_train set that consists
        of pseudo-labeled and labeled data.
        '''
        num_of_samples = int(len(self.test) * self.sample_rate)
        
        # Train the model and create the pseudo-labels
        self.model.fit(X, y)
        pseudo_labels = self.model.predict(self.test)
        
        # Add the pseudo-labels to the test set
        augmented_test = self.test.copy(deep=True)
        augmented_test[self.target] = pseudo_labels
        
        # Take a subset of the test set with pseudo-labels and append in onto
        # the training set
        sampled_test = augmented_test.sample(n=num_of_samples)
        temp_train = pd.concat([X, y], axis=1)
#         print(sampled_test.head())
#         print(temp_train.head())
        augmented_train = pd.concat([sampled_test, temp_train])
        augmented_train = shuffle(augmented_train)
        augmented_X = augmented_train.drop(['state'], axis=1)
        augmented_y = augmented_train['state']
        
        return augmented_X, augmented_y
    
    def predict(self, X):
        return self.model.predict(X)
    
    def get_model_name(self):
        return self.model.__class__.__name__

In [88]:
#Pseudo-labeling on actual test set
train_df = pd.read_csv('../data/processed/train.csv')
test_df = pd.read_csv('../data/processed/test.csv')

# Preprocess the data
X_train, X_test = train_df.drop(['state', 'name'], axis=1), test_df.drop(['state', 'name'], axis=1)
y_train, y_test = pd.DataFrame(train_df['state']),  pd.DataFrame(test_df['state'])

# Create the PseudoLabeler with XGBRegressor as the base regressor
model = PseudoLabeler(
#    RandomForestClassifier(n_estimators=100),
    GradientBoostingClassifier(n_estimators=100),
    X_test,
    'state',
    sample_rate=0.2
)

# Train the model and use it to predict
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

print(classification_report(y_test, y_hat))
print(confusion_matrix(y_test, y_hat))
accuracy = balanced_accuracy_score(y_test, y_hat)
print('balanced accuracy', accuracy)

(21418, 150)
(21418, 1)


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


            x0        y0        z0        x1        y1        z1        x2  \
5918  0.071289 -0.014648  1.001953  0.057617 -0.015625  0.992188  0.067383   
4354  0.079102 -0.003906  0.993164  0.070312 -0.004883  0.998047  0.075195   
3098  0.068359 -0.004883  1.000000  0.056641 -0.009766  0.990234  0.053711   
5199  0.055664 -0.009766  0.991211  0.055664 -0.013672  0.991211  0.064453   
2546  0.071289 -0.010742  0.990234  0.072266 -0.007812  0.991211  0.072266   

            y2        z2        x3  ...       x47       y47       z47  \
5918 -0.012695  0.990234  0.066406  ...  0.071289 -0.017578  1.000000   
4354 -0.013672  0.992188  0.076172  ...  0.082031 -0.008789  0.996094   
3098 -0.010742  0.983398  0.070312  ...  0.062500 -0.009766  0.986328   
5199 -0.011719  0.999023  0.066406  ...  0.066406 -0.014648  0.996094   
2546 -0.006836  0.991211  0.072266  ...  0.075195 -0.010742  0.985352   

           x48       y48       z48       x49       y49       z49  state  
5918  0.061523 -0.

In [137]:
#Incremental pseudo-labeling
#Case 0: add pseudo-labeled data incrementally, no discrimination. We do per subject
from sklearn.model_selection import LeaveOneGroupOut
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
import random

train_df = pd.read_csv('../data/processed/train.csv')
test_df = pd.read_csv('../data/processed/test.csv')

X_train, X_test = train_df.drop(['state', 'name'], axis=1), test_df.drop(['state', 'name'], axis=1)
y_train, y_test = pd.DataFrame(train_df['state']),  pd.DataFrame(test_df['state'])

groups_dict = train_df.groupby('name')

# Train initial model
index = np.random.randint(0,len(groups_dict), dtype='int')
subject_supervised = list(groups_dict.groups)[index]

model = PseudoLabeler(
#    RandomForestClassifier(n_estimators=100),
    GradientBoostingClassifier(n_estimators=100, n_jobs=-1),
    X_test,
    'state',
    sample_rate=0.0
)

X_supervised = pd.DataFrame(data=X_train.values[groups_dict.groups[subject_supervised]])
y_supervised = pd.DataFrame(data=y_train.values[groups_dict.groups[subject_supervised]], columns=['state'])

model.fit(X_supervised, y_supervised)
y_hat = model.predict(X_test)

print('_______initial supervised model______')
print(classification_report(y_test, y_hat))
print(confusion_matrix(y_test, y_hat))
accuracy = balanced_accuracy_score(y_test, y_hat)
print('balanced accuracy', accuracy)

for subject in groups_dict.groups:
    # The rest of the subjects for unsupervised learning
    if subject != subject_supervised:
        y_hat = model.predict()
        model.partial_fit()
     

(2989, 150)
(2989, 1)
sample rate: 0.0


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


_______initial supervised model______
              precision    recall  f1-score   support

           0       0.80      0.95      0.87      2566
           1       0.89      0.76      0.82      2661
           2       0.68      0.38      0.49       395
           3       0.37      0.40      0.38       411

   micro avg       0.79      0.79      0.79      6033
   macro avg       0.68      0.62      0.64      6033
weighted avg       0.80      0.79      0.79      6033

[[2442   34   12   78]
 [ 497 2034   25  105]
 [  66   76  150  103]
 [  62  151   33  165]]
balanced accuracy 0.6243141861932401
